In [ ]:
%pip install selenium

In [ ]:
# 오늘의 경기 승률 예측 코드

In [ ]:
# 4-1. 경기 정보 데이터 (팀, 구장, 날짜, 시간) 스크래핑
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Chrome 드라이버를 새로 설치하여 Chrome 실행
driver = webdriver.Chrome()

# 사이트 접속
driver.get("https://www.koreabaseball.com/Schedule/Schedule.aspx")

# 페이지가 로드될 때까지 대기
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
time.sleep(2)

# 텍스트가 '프리뷰'인 a태그 추출
# 해당 사이트는 동적 페이지이기 때문에 동적 웹 스크래핑 라이브러리인 selenium을 사용해야 함
a_tag_preview_text = driver.find_elements(By.XPATH, "//a[normalize-space(text())='프리뷰']")


# URL을 저장할 빈 리스트 생성
preview_url_list = []

# 프리뷰 URL 추출
for index, link in enumerate(a_tag_preview_text):
    href = link.get_attribute('href')
    preview_url_list.append(href)

# Chrome 닫기
driver.quit()

# 출력
print(preview_url_list)

In [ ]:
# 4-2. 팀 기록 데이터 (순위, 팀명, 타율, 경기, 타석, 타수, 득점, 안타, 2루타, 3루타, 홈런, 투타, 타점, 희생번트, 희생플라이) 스크래핑

import requests
import pandas as pd
from bs4 import BeautifulSoup

url = "https://www.koreabaseball.com/Record/Team/Hitter/Basic1.aspx"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36"
}

# url로 html을 paersing 함, headers는 사용자가 사람이라는 것을 인증하기 위함
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# <td> 태그 전체 수집
td_tags = soup.find_all('td')

# td에서 텍스트만 추출
td_text = [td.get_text(strip=True) for td in td_tags]

# 열 개수 (한 팀당 데이터 수)
# 순위 + 팀명 + AVG + G + PA + AB + R + H + 2B + 3B + HR + TB + RBI + SAC + SF
num_columns = 15

# td 리스트를 행 단위로 잘라서 리스트로 변환
data = [td_text[i:i + num_columns] for i in range(0, len(td_text), num_columns)]

# 열 이름 설정
columns = ['순위', '팀명', '타율', '경기', '타석', '타수', '득점', '안타', '2루타', '3루타', '홈런', '투타', '타점', '희생번트', '희생플라이']

# 데이터프레임 생성
team_info_df = pd.DataFrame(data, columns=columns)
team_info_df = team_info_df[team_info_df['순위'] != '합계']

# 출력
team_info_df

In [ ]:
# 4-3. 선수 기록 데이터 (순위, 선수명, 팀명, 타율, 경기, 타석, 타수, 득점, 안타, 2루타, 3루타, 홈런, 투타, 타점, 희생번트, 희생플라이) 스크래핑

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Chrome 브라우저 설정 (headless 모드도 가능)
options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # 브라우저 안 띄우려면 주석 해제
driver = webdriver.Chrome(options=options)

# KBO 타자 기록 페이지 열기
url = "https://www.koreabaseball.com/Record/Player/HitterBasic/Basic1.aspx"
driver.get(url)

# 대기 객체 (웹 요소가 로드될 때까지 대기)
wait = WebDriverWait(driver, 10)

# 데이터 저장 리스트
all_rows = []

# 페이지 번호가 1부터 시작한다고 가정
for page in range(1, 4):  # 예: 1~5페이지 수집
    print(f"{page}페이지 수집 중...")

    # 테이블이 로드될 때까지 대기
    wait.until(EC.presence_of_element_located((By.TAG_NAME, "table")))

    # 현재 페이지의 테이블에서 tr 추출
    table = driver.find_element(By.TAG_NAME, "tbody")
    rows = table.find_elements(By.TAG_NAME, "tr")

    # 테이블에서 각 행의 데이터를 추출
    for row in rows:
        tds = row.find_elements(By.TAG_NAME, "td")
        row_data = [td.text for td in tds]
        if row_data:  # 빈 줄 제외
            all_rows.append(row_data)

    # 다음 페이지로 넘어가기
    try:
        next_page_btn = driver.find_element(By.LINK_TEXT, str(page + 1))  # 페이지 버튼을 클릭
        next_page_btn.click()
        time.sleep(1.5)
    except:
        print(f"{page + 1} 페이지 없음.")
        break

# 열 이름 정의
columns = ['순위', '선수명', '팀명', '타율', '경기', '타석', '타수', '득점', '안타', '2루타', '3루타', '홈런', '투타', '타점', '희생번트', '희생플라이']

# DataFrame 생성
df = pd.DataFrame(all_rows, columns=columns)

# "합계" 행 제거
df = df[df['순위'] != '합계']


# 브라우저 종료
driver.quit()

# 결과 출력
df


1페이지 수집 중...
2페이지 수집 중...
3페이지 수집 중...
4 페이지 없음.


,순위,선수명,팀명,타율,경기,타석,타수,득점,안타,2루타,3루타,홈런,투타,타점,희생번트,희생플라이
0,1,전민재,롯데,0.407,19,63,54,9,22,5,0,0,27,5,4,0
1,2,손아섭,NC,0.389,16,60,54,9,21,2,2,0,27,10,0,1
2,3,문보경,LG,0.364,18,79,66,16,24,3,0,4,39,18,0,2
3,4,강민호,삼성,0.354,18,72,65,7,23,7,0,0,30,13,0,0
4,5,김현수,LG,0.353,18,58,51,7,18,3,0,1,24,9,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,58,로하스,KT,0.197,18,81,66,10,13,3,0,1,19,4,0,1
58,59,장성우,KT,0.193,17,67,57,5,11,2,0,1,16,10,0,2
59,60,구자욱,삼성,0.189,19,84,74,14,14,4,0,4,30,14,1,1
60,61,정준재,SSG,0.180,15,61,50,9,9,1,1,0,12,3,2,0
